# ImageRearranger_minimal Jupyter Notebook

A minimal version of [ImageRearranger.ipynb](ImageRearranger.ipynb). This notebook:

* Loads a collection of images in the form of a single mosaic image.
* Computes high-dimensional features which describe the images, using an image pyramid
* Reduces the dimensionality of those features to a 2D point cloud, using UMAP
* Rectifies the 2D point cloud into a grid, producing an ordered mosaic of the input images.

---

### Instructions

* For complete installation and deployment instructions, see [ImageRearranger.ipynb](ImageRearranger.ipynb). 
* Because this notebook requires the installation of numerous Python libraries, *working in a virtual environment is strongly recommended*.

---

### Credits

* Based on Kyle McDonald's [ImageRearranger](https://github.com/kylemcdonald/ImageRearranger/tree/master?tab=readme-ov-file).
* Includes code from Kyle McDonald's [python-utils](https://github.com/kylemcdonald/python-utils) repository. 
* Inspired by [this collection](https://twitter.com/JUSTIN_CYR/status/829196024631681024) of pixel art by Justin Cyr.
* Updates & Extensions by Golan Levin, February 2025


---
# Settings, Imports, and Definitions

In [ ]:
# -----------------------------------------------
# GLOBAL SETTINGS

IMAGE_FILENAME = "inputs/src_cyr_32.png" # a pre-made mosaic image.
TILE_SIZE = 32 # Define tile size for image processing


In [ ]:
# -------------------------------
# NOTEBOOK SETUP

# Enable inline plotting for Jupyter notebooks
%matplotlib inline

# Import necessary libraries
import os
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import IPython
from umap import UMAP

# Improve matplotlib rendering quality for retina displays
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

# Improve image rendering in Jupyter
IPython.core.display.display_html(
    IPython.core.display.HTML("<style>img{image-rendering: pixelated}</style>")
)

In [ ]:
# -------------------------------
# IMAGE UTILITY FUNCTIONS

# Reads an image from a file using OpenCV.
# Returns the loaded image as a NumPy array.
def imread(filename, mode=None):
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Image file '{filename}' not found.")
    if mode == 'rgb':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif mode == 'gray':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


# Displays an image (a NumPy array) in a Jupyter notebook using Matplotlib.
# Increases display resolution if `retina` is True.
def imshow(img, retina=False):
    if img is None:
        raise ValueError("Input image is None.")
    plt.figure(figsize=(6, 6) if retina else (4, 4))
    cmap = 'gray' if len(img.shape) == 2 else None
    plt.imshow(img, cmap=cmap)
    plt.axis('off')
    plt.show()

# -------------------------------
# IMAGE MOSAIC FUNCTIONS

# Swap axes of an array (supports NumPy and PyTorch).
def swapaxes(x, a, b):
    try:
        return x.swapaxes(a, b)
    except AttributeError:  # Support PyTorch tensors
        return x.transpose(a, b)


# Arranges a batch of images into a mosaic grid.
def make_mosaic(x, nx=None, ny=None):
    """
    Parameters:
        x (np.ndarray): Image batch of shape (n, h, w) or (n, h, w, c).
        nx (int): Number of columns in the mosaic.
        ny (int): Number of rows in the mosaic.
    Returns:
        np.ndarray: Mosaic image.
    """
    if not isinstance(x, np.ndarray):
        x = np.asarray(x)

    n, h, w = x.shape[:3]
    has_channels = len(x.shape) > 3
    c = x.shape[3] if has_channels else None

    if nx is None and ny is None:
        ny = int(np.sqrt(n))  # Default to a roughly square layout
        nx = (n + ny - 1) // ny  # Ensure enough columns
    elif ny is None:
        ny = n // nx
    elif nx is None:
        nx = n // ny

    end_shape = (w, c) if has_channels else (w,)
    mosaic = x.reshape(ny, nx, h, *end_shape)
    mosaic = swapaxes(mosaic, 1, 2)
    mosaic = mosaic.reshape(ny * h, nx * w, c) if has_channels else mosaic.reshape(ny * h, nx * w)

    return mosaic


# Splits a mosaic image into individual images.
# Note: `//` is the Python floored-division operator. 
def unmake_mosaic(mosaic, nx=None, ny=None, w=None, h=None):
    """
    Parameters:
        mosaic (np.ndarray): The mosaic image.
        nx (int): Number of images per row.
        ny (int): Number of images per column.
        w (int): Width of each sub-image.
        h (int): Height of each sub-image.
    Returns:
        np.ndarray: Array of split images.
    """
    hh, ww = mosaic.shape[:2]

    if nx is not None or ny is not None:
        if nx is None:
            h = hh // ny
            w = h
            nx = ww // w
        elif ny is None:
            w = ww // nx
            h = w
            ny = hh // h
        else:
            w = ww // nx
            h = hh // ny
    elif w is not None or h is not None:
        if w is None:
            w = h
        elif h is None:
            h = w
        nx = ww // w
        ny = hh // h

    end_shape = (w, mosaic.shape[2]) if len(mosaic.shape) > 2 else (w,)

    x = mosaic.reshape(ny, h, nx, *end_shape)
    x = swapaxes(x, 1, 2)
    x = x.reshape(-1, h, *end_shape)

    return x


# -------------------------------
# IMAGE PLOTTING FUNCTIONS

# Places images at given (x, y) coordinates onto a blank canvas.
def plot_images(images, xy, blend=np.maximum, canvas_shape=(512, 512), fill=0):
    """
    Parameters:
        images (np.ndarray): Array of images to place.
        xy (np.ndarray): (x, y) positions for each image.
        blend (func): Blending function for overlapping images (default: np.maximum).
        canvas_shape (tuple): Shape of output canvas (height, width, channels).
        fill (int): Fill value for empty pixels (default: 0).
    Returns:
        np.ndarray: Final composed image.
    """
    h, w = images.shape[1:3]
    if images.ndim == 4:
        canvas_shape = (canvas_shape[0], canvas_shape[1], images.shape[3])

    min_xy = np.amin(xy, 0)
    max_xy = np.amax(xy, 0)

    min_canvas = np.array((0, 0))
    max_canvas = np.array((canvas_shape[0] - h, canvas_shape[1] - w))

    xy_mapped = min_canvas + (xy - min_xy) * (max_canvas - min_canvas) / (max_xy - min_xy)
    xy_mapped = xy_mapped.astype(int)

    canvas = np.full(canvas_shape, fill)
    for image, pos in zip(images, xy_mapped):
        x_off, y_off = pos
        sub_canvas = canvas[y_off:y_off+h, x_off:x_off+w]
        sub_image = image[:h, :w]
        try:
            canvas[y_off:y_off+h, x_off:x_off+w] = blend(sub_canvas, sub_image)
        except ValueError:
            print(pos, h, w, min_canvas, max_canvas)
            raise

    return canvas

# 1. Load a collection of images as a single mosaic image.

In [ ]:
# -----------------------------------
# LOAD IMAGE DATA (MOSAIC OR FOLDER)

import shutil

# Load full mosaic image
print(f"Loading mosaic image: {IMAGE_FILENAME}")
img = imread(IMAGE_FILENAME, 'rgb')  
imshow(img, retina=True)

# Explode mosaic image into individual tiles
images = unmake_mosaic(img, w=TILE_SIZE)
n_loaded_images = len(images)  # Should be `nx * ny`

# Compute number of columns (nx) and rows (ny) in the mosaic
nx = img.shape[1] // TILE_SIZE  # Number of tiles per row
ny = img.shape[0] // TILE_SIZE  # Number of tiles per column

# Print the total number of extracted images
print(f"✅ Processing {len(images)} images, each of size {TILE_SIZE}×{TILE_SIZE} pixels.")
print(f"🟢 Mosaic dimensions: {nx}×{ny} tiles ({img.shape[1]}×{img.shape[0]} pixels).")


# 2. Compute an image pyramid

In [ ]:
# -----------------------------------
# FUNCTION TO MAKE BLURRED IMAGE SETS

# skimage.filters is imported here because 
# it's only used in this one place. 
from skimage.filters import gaussian

# Applies Gaussian blur to a batch of images.
def build_blurred(images, sigma):
    # Parameters:
    #    images (np.ndarray): Batch of images.
    #    sigma (float): Standard deviation for Gaussian kernel.
    # Returns:
    #    np.ndarray: Blurred images in uint8 format.
    blurred = []
    for image in images:
        # Apply Gaussian blur (output is normalized to [0, 1])
        blurred_image = gaussian(image, sigma=sigma, channel_axis=-1)
        
        # Scale back to [0, 255] and convert to uint8
        blurred_image = np.clip(blurred_image * 255, 0, 255).astype(np.uint8)
        blurred.append(blurred_image)

    return np.array(blurred)

# -----------------------------------------------
# GENERATE BLURRED IMAGE SETS
# We apply different levels of blur to each image in the dataset.
# The goal is to create a "multi-scale" representation for UMAP.
# Higher `sigma` values result in more aggressive blurring.

nx = int(img.shape[1] / TILE_SIZE)  # Number of columns
ny = int(img.shape[0] / TILE_SIZE)  # Number of rows
blur_levels = [4, 2, 1]    # Different blur intensities

# Apply Gaussian blur at different levels and visualize the results
blurred_0 = build_blurred(images, blur_levels[0])
imshow(make_mosaic(blurred_0, nx=nx, ny=ny), retina=True)  # Most blurred

blurred_1 = build_blurred(images, blur_levels[1])
imshow(make_mosaic(blurred_1, nx=nx, ny=ny), retina=True)  # Medium blur

blurred_2 = build_blurred(images, blur_levels[2])
imshow(make_mosaic(blurred_2, nx=nx, ny=ny), retina=True)  # Light blur

In [ ]:
# -----------------------------------
# CONSTRUCT THE IMAGE FEATURE PYRAMID

# Number of individual image patches
n = len(images)

# ⚡ Feature Stacking:
# We create a feature matrix (`pyr`) where each row represents an image,
# and each column contains pixel values from different levels of blurring.
# This allows UMAP (in the next step) to learn a representation 
# that takes multiple scales of detail into account.

pyr = np.hstack((
    blurred_0.reshape(n, -1),  # Strongly blurred images
    blurred_1.reshape(n, -1),  # Moderately blurred images
    blurred_2.reshape(n, -1),  # Lightly blurred images
    images.reshape(n, -1)      # Original images
))

# ✅ After this step:
# `pyr` is now a 2D array of shape (n, feature_dim), where:
#   - `n` is the number of image patches.
#   - `feature_dim` is the combined size of all image versions.
# This feature representation will be used as input to UMAP in the next step.

# ---------------------------------------
# Uncomment one of the lines below to experiment with different sources:
# source = images     # Use original images
# source = blurred_0  # Use blurred images
source = pyr          # Default: Use the image pyramid (multi-scale features)


---
# 3. Dimensionality Reduction with UMAP

In [ ]:
# ---------------------------------------
# APPLY UMAP FOR DIMENSIONALITY REDUCTION
# Reduces high-dimensional image data into a 2D space.
# Note: This cell can take some time — e.g. ~5s for 1K points

# ---------------------------------------
# 1️⃣ Import necessary libraries
from umap import UMAP

# ---------------------------------------
# 2️⃣ CONFIGURE UMAP PARAMETERS
# You can also add `random_state=12345` to set the UMAP random seed to 12345.

reducer = UMAP(
    n_neighbors=15,     # Larger values (e.g., 50) preserve more global structure
    min_dist=0.1,       # Controls how tightly points are packed
    n_components=2,     # Number of output dimensions (keep at 2D)
    metric="euclidean"  # Other options: "cosine", "manhattan", "correlation"
)
print(f"⚠️ Note: UMAP may trigger a FutureWarning; you can safely ignore this.")

# ---------------------------------------
# 3️⃣ RUN UMAP OR T-SNE & GENERATE EMBEDDING
# An "embedding" is a lower-dimensional representation of the data, 
# where similar images are placed closer together in the new space.

%time Y = reducer.fit_transform(source.reshape(images.shape[0], -1).astype(np.float64))

# ---------------------------------------
# 4️⃣ Normalize Output to the range [0,1] for visualization
Y -= Y.min(axis=0)
Y /= Y.max(axis=0)


In [ ]:
canvas = plot_images(images, Y, canvas_shape=(2048, 2048, 3), blend=np.minimum, fill=255)
imshow(canvas, retina=True)

---
# 4. Rectify the 2D Embedding to a Grid

In [ ]:
# -----------------------------
# SOLVE THE ASSIGNMENT PROBLEM: 
# MAPPING UMAP POINTS TO A GRID

from lap import lapjv  # Linear Assignment Problem (LAP) solver
from scipy.spatial.distance import cdist  # Computes pairwise distances

# 1️⃣ Define the target grid dimensions
nx = int(img.shape[1] / TILE_SIZE) # Number of grid columns
ny = int(img.shape[0] / TILE_SIZE) # Number of grid rows

# 2️⃣ Generate a uniform 2D grid of (x, y) target positions
xv, yv = np.meshgrid(np.linspace(0, 1, nx), np.linspace(0, 1, ny))  # Evenly spaced grid
grid = np.dstack((xv, yv)).reshape(-1, 2)  # Convert to list of 2D points

# 3️⃣ Compute the pairwise squared Euclidean distance between UMAP points and grid points
#    - `Y` contains the 2D coordinates of images in the UMAP space.
#    - `grid` contains the target positions in a structured layout.
#    - `cdist()` calculates all pairwise distances.
%time cost = cdist(grid, Y, 'sqeuclidean')  # (num_grid_points, num_images)

# 4️⃣ Convert cost matrix to integer values (LAP solver works faster with integers)
cost = cost * (100000. / cost.max())  # Scale costs to large integers
cost = cost.astype(int)  # Convert to integer type

# 5️⃣ Solve the Linear Assignment Problem (LAP) using Jonker-Volgenant Algorithm
# See: https://en.wikipedia.org/wiki/Hungarian_algorithm. 
# Here we compute the optimal assignment, and print the execution time 
# Note: this implementation sorts black padding squares, technically incorrect. 
totalDataPoints = nx * ny  # Number of points to assign
%time min_cost, row_assigns, col_assigns = lapjv(cost, extend_cost=True)

# 6️⃣ Map UMAP positions (`Y`) to the nearest grid positions (`grid_jv`)
grid_jv = grid[col_assigns[:totalDataPoints]]

# 7️⃣ Visualization: Draw arrows from original UMAP positions to assigned grid positions
plt.figure(figsize=(8, 8))
for start, end in zip(Y, grid_jv):
    plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
              head_length=0.01, head_width=0.01)
plt.show()


In [ ]:
# ------------------------------
# RECONSTRUCT THE ORDERED MOSAIC

# 1️⃣ Swap x and y coordinates to match the (row, column) ordering of the grid
grid_tuples = [(y, x) for (x, y) in map(tuple, grid_jv)]

# 2️⃣ Pair each image with its corresponding grid position
image_grid_pairs = list(zip(grid_tuples, images))

# 3️⃣ Sort the image-grid pairs by grid position (top-left to bottom-right)
sorted_pairs = sorted(image_grid_pairs)  # Sorts by (y, x) automatically

# 4️⃣ Extract the images from the sorted pairs (now correctly ordered for the mosaic).
# Note that the position data is now ignored.
sorted_images = [image for (position, image) in sorted_pairs]

# 5️⃣ Arrange sorted images into a final mosaic nparray
mosaic = make_mosaic(sorted_images, nx=nx, ny=ny)

# 6️⃣ Display the final, neatly arranged mosaic (TRUE SIZE)
# Note: `imshow(mosaic, retina=True)` is not pixel-perfect.
import PIL.Image
from IPython.display import display, HTML
# Save the mosaic as an image file
output_filename = "final_mosaic.png"
PIL.Image.fromarray(mosaic).save(output_filename)

# 7️⃣ (Force Jupyter to) display the image at its true size using HTML
# Otherwise Jupyter may resize the image and add a dumb border. 
display(HTML(f'<img src="{output_filename}" width="{mosaic.shape[1]}" height="{mosaic.shape[0]}" style="border:0px;">'))
print(f"✅ Generated UMAP mosaic with dimensions: {mosaic.shape[1]} x {mosaic.shape[0]} pixels")
